In [34]:
from math import log
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [2]:
def calculate_bic(n, mse, num_params):
	bic = n * log(mse) + num_params * log(n)
	return bi


#df=pd.read_csv(r"E:\copula\nwm_outputs\conus\tau_table_stat_sig.csv",index_col=0)
#df=pd.read_csv(r"E:\copula\nwm_outputs\tau_table_stat_sig_all.csv",index_col=0)
df=pd.read_csv(r"E:\copula\nwm_outputs\tau_table_15day_stat_sig_CONUS.csv",index_col=0)
df.VPUID=df.VPUID.astype('str') # large file so sometimes it does read it as strings
df.RPUID=df.RPUID.astype('str')

c:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
class Regression:
    def __init__(self,dataframe,stream,type_of_regress):
        self.df=dataframe
        self.stream=stream
        self.func=type_of_regress
    
    def data(self):
        x=self.df["DRAIN_ratio"]
        if self.stream == "POM":
            y=self.df['POM_tau']
        elif self.stream == "POT":
            y=self.df['POT_tau']
        return x,y
    def regress(self):
        x,y=self.data()
        if self.func == "Log" :
            x=np.log(x)
        elif self.func == "Power":
            x=np.log(x)
            y=np.log(y)
        
        model=LinearRegression()
        model.fit(x.values.reshape(-1,1), y.values)
        yhat=model.predict(x.values.reshape(-1,1))
        mse=mean_squared_error(y.values,yhat)
        r2=r2_score(y.values,yhat)
        
        if self.func =="Log" :
            print("Best fit line is:   tau=%.3f *ln(Ra) + %.3f" %(model.coef_[0],model.intercept_))
            print(" R_squared : %.4f  , MSE : %.4f " %(r2,mse))
        elif self.func == "Power":
            print("Best fit line is:   tau=%.3f * Ra^(%.3f)"  %(np.exp(model.intercept_), model.coef_[0],))
            print(" R_squared : %.4f  , MSE : %.4f " %(r2,mse))
        out={'Coeff' : model.coef_,
             'Intercept'  : model.intercept_,
             'MSE': mse,
             'R_squared': r2,
             'predicted' : yhat}
        
        return out


In [61]:
class Regression_ols:
    def __init__(self,dataframe,stream,type_of_regress):
        self.df=dataframe
        self.stream=stream
        self.func=type_of_regress
    
    def data(self):
        x=self.df["DRAIN_ratio"]
        if self.stream == "POM":
            y=self.df['POM_tau']
        elif self.stream == "POT":
            y=self.df['POT_tau']
        return x,y
    def regress(self):
        x,y=self.data()
        if self.func == "Log" :
            x=np.log(x)
        elif self.func == "Power":
            x=np.log(x)
            y=np.log(y)
        
        x=sm.add_constant(x)
        model=sm.OLS(y, x).fit()

        if self.func =="Log" :
            print("Best fit line is:   tau=%.3f *ln(Ra) + %.3f" %(model.params[1],model.params[0]))
            print(" R_squared : %.4f  , MSE : %.4f " %(model.rsquared,model.mse_resid))
        elif self.func == "Power":
            print("Best fit line is:   tau=%.3f * Ra^(%.3f)"  %(np.exp(model.params[1]), model.params[1],))
            print(" R_squared : %.4f  , MSE : %.4f " %(model.rsquared,model.mse_resid))
        out={'Coeff' : model.params[1],
             'Intercept'  : model.params[0],
             'MSE': model.mse_resid,
             'R_squared': model.rsquared,
             'p_value' : model.pvalues[1]}
        
        return out


In [62]:

r2_pom=[]
ms_error_pom=[]
equation_pom=[]
coef_pom=[]
intercept_pom=[]
r2_pot=[]
ms_error_pot=[]
equation_pot=[]
coef_pot=[]
intercept_pot=[]
idx=[]

## doing this for conus first
a=Regression(dataframe=df,stream='POM',type_of_regress='Log').regress()
b=Regression(dataframe=df,stream='POT',type_of_regress='Log').regress()
equation_pom.append("tau=%.3f *ln(Ra) + %.3f" %(a['Coeff'],a['Intercept']))
equation_pot.append("tau=%.3f *ln(Ra) + %.3f" %(b['Coeff'],b['Intercept']))
r2_pom.append(a['R_squared'])
r2_pot.append(b['R_squared'])
ms_error_pom.append(a['MSE'])
ms_error_pot.append(b['MSE'])
coef_pom.append(a['Coeff'][0])
coef_pot.append(b['Coeff'][0])
intercept_pom.append(a['Intercept'])
intercept_pot.append(b['Intercept'])
idx.append('CONUS')

## for individual hu_ids
huc_list=df.VPUID.unique()
for huc_id in huc_list:
    a=Regression(dataframe=df[df.VPUID==huc_id],stream='POM',type_of_regress='Log').regress()
    b=Regression(dataframe=df[df.VPUID==huc_id],stream='POT',type_of_regress='Log').regress()
    equation_pom.append("tau=%.3f *ln(Ra) + %.3f" %(a['Coeff'],a['Intercept']))
    equation_pot.append("tau=%.3f *ln(Ra) + %.3f" %(b['Coeff'],b['Intercept']))
    r2_pom.append(a['R_squared'])
    r2_pot.append(b['R_squared'])
    ms_error_pom.append(a['MSE'])
    ms_error_pot.append(b['MSE'])
    coef_pom.append(a['Coeff'][0])
    coef_pot.append(b['Coeff'][0])
    intercept_pom.append(a['Intercept'])
    intercept_pot.append(b['Intercept'])
    idx.append(huc_id)

df_regress=pd.DataFrame({'HUC_ID':idx,
                        'R_sq_POM':r2_pom,
                        'MSE_POM':ms_error_pom,
                        'Eq_POM':equation_pom,
                        'R_sq_POT':r2_pot,
                        'MSE_POT':ms_error_pot,
                        'Eq_POT':equation_pot,
                        'Coeff_POM':coef_pom,
                        'Coeff_POT':coef_pot,
                        'Intercept_POM':intercept_pom,
                        'Intercept_POT':intercept_pot})

Best fit line is:   tau=-0.055 *ln(Ra) + 0.802
 R_squared : 0.3570  , MSE : 0.0184 
Best fit line is:   tau=-0.049 *ln(Ra) + 0.823
 R_squared : 0.3340  , MSE : 0.0164 
Best fit line is:   tau=-0.053 *ln(Ra) + 0.756
 R_squared : 0.3500  , MSE : 0.0150 
Best fit line is:   tau=-0.050 *ln(Ra) + 0.756
 R_squared : 0.3352  , MSE : 0.0145 
Best fit line is:   tau=-0.057 *ln(Ra) + 0.798
 R_squared : 0.4262  , MSE : 0.0142 
Best fit line is:   tau=-0.058 *ln(Ra) + 0.804
 R_squared : 0.4449  , MSE : 0.0136 
Best fit line is:   tau=-0.057 *ln(Ra) + 0.818
 R_squared : 0.4439  , MSE : 0.0130 
Best fit line is:   tau=-0.056 *ln(Ra) + 0.821
 R_squared : 0.4387  , MSE : 0.0130 
Best fit line is:   tau=-0.056 *ln(Ra) + 0.852
 R_squared : 0.4555  , MSE : 0.0121 
Best fit line is:   tau=-0.052 *ln(Ra) + 0.849
 R_squared : 0.4347  , MSE : 0.0113 
Best fit line is:   tau=-0.062 *ln(Ra) + 0.859
 R_squared : 0.5079  , MSE : 0.0115 
Best fit line is:   tau=-0.061 *ln(Ra) + 0.860
 R_squared : 0.5099  , MSE : 

In [63]:
df_regress

,HUC_ID,R_sq_POM,MSE_POM,Eq_POM,R_sq_POT,MSE_POT,Eq_POT,Coeff_POM,Coeff_POT,Intercept_POM,Intercept_POT
0,CONUS,0.356980,0.018409,tau=-0.055 *ln(Ra) + 0.802,0.333998,0.016351,tau=-0.049 *ln(Ra) + 0.823,-0.054686,-0.048985,0.802023,0.822708
1,01,0.349987,0.015037,tau=-0.053 *ln(Ra) + 0.756,0.335207,0.014455,tau=-0.050 *ln(Ra) + 0.756,-0.052918,-0.050208,0.756004,0.756435
2,02,0.426185,0.014215,tau=-0.057 *ln(Ra) + 0.798,0.444882,0.013626,tau=-0.058 *ln(Ra) + 0.804,-0.056586,-0.057549,0.798377,0.804193
3,03N,0.443864,0.013010,tau=-0.057 *ln(Ra) + 0.818,0.438733,0.012968,tau=-0.056 *ln(Ra) + 0.821,-0.057073,-0.056391,0.817882,0.821169
4,03S,0.455451,0.012076,tau=-0.056 *ln(Ra) + 0.852,0.434738,0.011253,tau=-0.052 *ln(Ra) + 0.849,-0.056409,-0.052215,0.852203,0.849301
5,03W,0.507872,0.011500,tau=-0.062 *ln(Ra) + 0.859,0.509874,0.011081,tau=-0.061 *ln(Ra) + 0.860,-0.061777,-0.060883,0.858957,0.860276
6,04,0.267460,0.014927,tau=-0.048 *ln(Ra) + 0.763,0.301711,0.014687,tau=-0.051 *ln(Ra) + 0.771,-0.047542,-0.051301,0.763210,0.770902
7,05,0.537236,0.011639,tau=-0.062 *ln(Ra) + 0.841,0.550395,0.012423,tau=-0.065 *ln(Ra) + 0.845,-0.061612,-0.065362,0.841287,0.845408
8,06,0.450729,0.009662,tau=-0.059 *ln(Ra) + 0.874,0.433034,0.009064,tau=-0.056 *ln(Ra) + 0.875,-0.059413,-0.055516,0.873953,0.874813
9,07,0.504214,0.012010,tau=-0.056 *ln(Ra) + 0.842,0.491070,0.011741,tau=-0.054 *ln(Ra) + 0.843,-0.055760,-0.053702,0.841801,0.842812


In [47]:
x=np.log(df[df.VPUID==huc_id]["DRAIN_ratio"])
y=(df[df.VPUID==huc_id]['POM_tau'])
model=LinearRegression()
model.fit(x.values.reshape(-1,1), y.values)
print(model.coef_)
print(model.intercept_)
print(model.score(x.values.reshape(-1,1), y.values))

[-0.0431693]
0.8075206248015765
0.20050580444818888


In [50]:
x=sm.add_constant(x)

In [51]:
model1 = sm.OLS(y, x).fit()

In [52]:
model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                POM_tau   R-squared:                       0.201
Model:                            OLS   Adj. R-squared:                  0.200
Method:                 Least Squares   F-statistic:                     1756.
Date:                Tue, 04 Jun 2024   Prob (F-statistic):               0.00
Time:                        15:34:39   Log-Likelihood:                 4030.5
No. Observations:                7004   AIC:                            -8057.
Df Residuals:                    7002   BIC:                            -8043.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.8075      0.003    319.543      0.000       0.803       0.812
DRAIN_ratio    -0.0432      0.001    -41.905      0.000      -0.045      -0.041
==============================================================================
Omnibus:                     1059.443   Durbin-Watson:                   1.447
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1669.274
Skew:                          -1.053   Prob(JB):                         0.00
Kurtosis:                       4.135   Cond. No.                         4.21
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [65]:

r2_pom=[]
ms_error_pom=[]
equation_pom=[]
coef_pom=[]
intercept_pom=[]
p_value_pom=[]
r2_pot=[]
ms_error_pot=[]
equation_pot=[]
coef_pot=[]
intercept_pot=[]
p_value_pot=[]
idx=[]

## doing this for conus first
a=Regression_ols(dataframe=df,stream='POM',type_of_regress='Log').regress()
b=Regression_ols(dataframe=df,stream='POT',type_of_regress='Log').regress()
equation_pom.append("tau=%.3f *ln(Ra) + %.3f" %(a['Coeff'],a['Intercept']))
equation_pot.append("tau=%.3f *ln(Ra) + %.3f" %(b['Coeff'],b['Intercept']))
r2_pom.append(a['R_squared'])
r2_pot.append(b['R_squared'])
ms_error_pom.append(a['MSE'])
ms_error_pot.append(b['MSE'])
coef_pom.append(a['Coeff'])
coef_pot.append(b['Coeff'])
intercept_pom.append(a['Intercept'])
intercept_pot.append(b['Intercept'])
p_value_pom.append(a['p_value'])
p_value_pot.append(b['p_value'])
idx.append('CONUS')

## for individual hu_ids
huc_list=df.VPUID.unique()
for huc_id in huc_list:
    a=Regression_ols(dataframe=df[df.VPUID==huc_id],stream='POM',type_of_regress='Log').regress()
    b=Regression_ols(dataframe=df[df.VPUID==huc_id],stream='POT',type_of_regress='Log').regress()
    equation_pom.append("tau=%.3f *ln(Ra) + %.3f" %(a['Coeff'],a['Intercept']))
    equation_pot.append("tau=%.3f *ln(Ra) + %.3f" %(b['Coeff'],b['Intercept']))
    r2_pom.append(a['R_squared'])
    r2_pot.append(b['R_squared'])
    ms_error_pom.append(a['MSE'])
    ms_error_pot.append(b['MSE'])
    coef_pom.append(a['Coeff'])
    coef_pot.append(b['Coeff'])
    intercept_pom.append(a['Intercept'])
    intercept_pot.append(b['Intercept'])
    p_value_pom.append(a['p_value'])
    p_value_pot.append(b['p_value'])
    idx.append(huc_id)

df_regress=pd.DataFrame({'HUC_ID':idx,
                        'R_sq_POM':r2_pom,
                        'MSE_POM':ms_error_pom,
                        'Eq_POM':equation_pom,
                        'R_sq_POT':r2_pot,
                        'MSE_POT':ms_error_pot,
                        'Eq_POT':equation_pot,
                        'Coeff_POM':coef_pom,
                        'Coeff_POT':coef_pot,
                        'Intercept_POM':intercept_pom,
                        'Intercept_POT':intercept_pot,
                        'p_vale_POM': p_value_pom,
                        'p_value_POT': p_value_pot})

Best fit line is:   tau=-0.055 *ln(Ra) + 0.802
 R_squared : 0.3570  , MSE : 0.0184 
Best fit line is:   tau=-0.049 *ln(Ra) + 0.823
 R_squared : 0.3340  , MSE : 0.0164 
Best fit line is:   tau=-0.053 *ln(Ra) + 0.756
 R_squared : 0.3500  , MSE : 0.0150 
Best fit line is:   tau=-0.050 *ln(Ra) + 0.756
 R_squared : 0.3352  , MSE : 0.0145 
Best fit line is:   tau=-0.057 *ln(Ra) + 0.798
 R_squared : 0.4262  , MSE : 0.0142 
Best fit line is:   tau=-0.058 *ln(Ra) + 0.804
 R_squared : 0.4449  , MSE : 0.0136 
Best fit line is:   tau=-0.057 *ln(Ra) + 0.818
 R_squared : 0.4439  , MSE : 0.0130 
Best fit line is:   tau=-0.056 *ln(Ra) + 0.821
 R_squared : 0.4387  , MSE : 0.0130 
Best fit line is:   tau=-0.056 *ln(Ra) + 0.852
 R_squared : 0.4555  , MSE : 0.0121 
Best fit line is:   tau=-0.052 *ln(Ra) + 0.849
 R_squared : 0.4347  , MSE : 0.0113 
Best fit line is:   tau=-0.062 *ln(Ra) + 0.859
 R_squared : 0.5079  , MSE : 0.0115 
Best fit line is:   tau=-0.061 *ln(Ra) + 0.860
 R_squared : 0.5099  , MSE : 

In [66]:
df_regress

,HUC_ID,R_sq_POM,MSE_POM,Eq_POM,R_sq_POT,MSE_POT,Eq_POT,Coeff_POM,Coeff_POT,Intercept_POM,Intercept_POT,p_vale_POM,p_value_POT
0,CONUS,0.356980,0.018409,tau=-0.055 *ln(Ra) + 0.802,0.333998,0.016352,tau=-0.049 *ln(Ra) + 0.823,-0.054686,-0.048985,0.802023,0.822708,0.000000e+00,0.000000e+00
1,01,0.349987,0.015047,tau=-0.053 *ln(Ra) + 0.756,0.335207,0.014464,tau=-0.050 *ln(Ra) + 0.756,-0.052918,-0.050208,0.756004,0.756435,4.226039e-288,3.548285e-273
2,02,0.426185,0.014219,tau=-0.057 *ln(Ra) + 0.798,0.444882,0.013630,tau=-0.058 *ln(Ra) + 0.804,-0.056586,-0.057549,0.798377,0.804193,0.000000e+00,0.000000e+00
3,03N,0.443864,0.013014,tau=-0.057 *ln(Ra) + 0.818,0.438733,0.012972,tau=-0.056 *ln(Ra) + 0.821,-0.057073,-0.056391,0.817882,0.821169,0.000000e+00,0.000000e+00
4,03S,0.455451,0.012086,tau=-0.056 *ln(Ra) + 0.852,0.434738,0.011262,tau=-0.052 *ln(Ra) + 0.849,-0.056409,-0.052215,0.852203,0.849301,0.000000e+00,5.664844e-304
5,03W,0.507872,0.011505,tau=-0.062 *ln(Ra) + 0.859,0.509874,0.011085,tau=-0.061 *ln(Ra) + 0.860,-0.061777,-0.060883,0.858957,0.860276,0.000000e+00,0.000000e+00
6,04,0.267460,0.014932,tau=-0.048 *ln(Ra) + 0.763,0.301711,0.014692,tau=-0.051 *ln(Ra) + 0.771,-0.047542,-0.051301,0.763210,0.770902,0.000000e+00,0.000000e+00
7,05,0.537236,0.011642,tau=-0.062 *ln(Ra) + 0.841,0.550395,0.012425,tau=-0.065 *ln(Ra) + 0.845,-0.061612,-0.065362,0.841287,0.845408,0.000000e+00,0.000000e+00
8,06,0.450729,0.009671,tau=-0.059 *ln(Ra) + 0.874,0.433034,0.009072,tau=-0.056 *ln(Ra) + 0.875,-0.059413,-0.055516,0.873953,0.874813,4.449738e-289,6.781294e-274
9,07,0.504214,0.012012,tau=-0.056 *ln(Ra) + 0.842,0.491070,0.011743,tau=-0.054 *ln(Ra) + 0.843,-0.055760,-0.053702,0.841801,0.842812,0.000000e+00,0.000000e+00


In [56]:
model1.pvalues[1]

0.0

In [23]:
df_regress=pd.DataFrame({'HUC_ID':idx,
                        'R_sq_POM':r2_pom,
                        'MSE_POM':ms_error_pom,
                        'Eq_POM':equation_pom,
                        'R_sq_POT':r2_pot,
                        'MSE_POT':ms_error_pot,
                        'Eq_POT':equation_pot,
                        'Coeff_POM':coef_pom,
                        'Coeff_POT':coef_pot,
                        'Intercept_POM':intercept_pom,
                        'Intercept_POT':intercept_pot})

In [25]:
df_regress

,HUC_ID,R_sq_POM,MSE_POM,Eq_POM,R_sq_POT,MSE_POT,Eq_POT,Coeff_POM,Coeff_POT,Intercept_POM,Intercept_POT
0,CONUS,0.356980,0.018409,tau=-0.055 *ln(Ra) + 0.802,0.333998,0.016351,tau=-0.049 *ln(Ra) + 0.823,-0.054686,-0.048985,0.802023,0.822708
1,01,0.349987,0.015037,tau=-0.053 *ln(Ra) + 0.756,0.335207,0.014455,tau=-0.050 *ln(Ra) + 0.756,-0.052918,-0.050208,0.756004,0.756435
2,02,0.426185,0.014215,tau=-0.057 *ln(Ra) + 0.798,0.444882,0.013626,tau=-0.058 *ln(Ra) + 0.804,-0.056586,-0.057549,0.798377,0.804193
3,03N,0.443864,0.013010,tau=-0.057 *ln(Ra) + 0.818,0.438733,0.012968,tau=-0.056 *ln(Ra) + 0.821,-0.057073,-0.056391,0.817882,0.821169
4,03S,0.455451,0.012076,tau=-0.056 *ln(Ra) + 0.852,0.434738,0.011253,tau=-0.052 *ln(Ra) + 0.849,-0.056409,-0.052215,0.852203,0.849301
5,03W,0.507872,0.011500,tau=-0.062 *ln(Ra) + 0.859,0.509874,0.011081,tau=-0.061 *ln(Ra) + 0.860,-0.061777,-0.060883,0.858957,0.860276
6,04,0.267460,0.014927,tau=-0.048 *ln(Ra) + 0.763,0.301711,0.014687,tau=-0.051 *ln(Ra) + 0.771,-0.047542,-0.051301,0.763210,0.770902
7,05,0.537236,0.011639,tau=-0.062 *ln(Ra) + 0.841,0.550395,0.012423,tau=-0.065 *ln(Ra) + 0.845,-0.061612,-0.065362,0.841287,0.845408
8,06,0.450729,0.009662,tau=-0.059 *ln(Ra) + 0.874,0.433034,0.009064,tau=-0.056 *ln(Ra) + 0.875,-0.059413,-0.055516,0.873953,0.874813
9,07,0.504214,0.012010,tau=-0.056 *ln(Ra) + 0.842,0.491070,0.011741,tau=-0.054 *ln(Ra) + 0.843,-0.055760,-0.053702,0.841801,0.842812
